In [6]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
import tensorflow as tf

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

2022-01-27 13:16:16.398677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 13:16:16.398725: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-01-27 13:16:25.211546: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-27 13:16:26.904310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 13:16:26.904475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 13:16:26.904559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dy

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6026418906247904325
 xla_global_id: -1]

asLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 13:16:26.904664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 13:16:26.904712: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 13:16:26.904758: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.1

In [8]:
#making_label_set

crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [9]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [10]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [11]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [12]:
@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [13]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [14]:
%%time
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize(img) for img in row_img]
train_x = np.array(dask.compute(*train))

CPU times: user 45.2 s, sys: 6.05 s, total: 51.3 s
Wall time: 14 s


In [55]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

CPU times: user 4.55 s, sys: 582 ms, total: 5.13 s
Wall time: 5.03 s


((5767,), (5767,), (5767,), (5767,))

In [16]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  
])
model.compile()

2022-01-27 13:17:56.378517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 13:17:56.378581: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


9420800/9406464 [==============================] - 0s 0us/step


In [17]:
%%time
batch = 100
data_length = len(train_x)
loop_num = int(np.floor(data_length/batch))

train_x_feature = dask.array.zeros((data_length,model(train_x[:1]).numpy().shape[1]))


count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:end_p] = model(train_x[start_p:end_p]).numpy()
        
    else:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:] = model(train_x[start_p:]).numpy()
            
    count += 1



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
CPU times: user 16min 6s, sys: 7min 5s, total: 23min 12s
Wall time: 1min 41s


## crop모델

In [40]:
# 파라미터 조정
y_all = label_crop
cnn_out_dim = 1280
description = crop_label_description
decoder = crop_label_decoder
nclass = len(description)

In [19]:
X, X_test, y, y_test = train_test_split(train_x_feature, y_all, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))

In [33]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:46253' processes=4 threads=16, memory=24.59 GiB>


[13:34:49] task [xgboost.dask]:tcp://127.0.0.1:40067 got new rank 0
[13:34:49] task [xgboost.dask]:tcp://127.0.0.1:35641 got new rank 1


[0]	train-mlogloss:1.14427
[1]	train-mlogloss:0.82625
[2]	train-mlogloss:0.61797
[3]	train-mlogloss:0.47364
[4]	train-mlogloss:0.36724
[5]	train-mlogloss:0.28829
[6]	train-mlogloss:0.22730
[7]	train-mlogloss:0.18021
[8]	train-mlogloss:0.14371
[9]	train-mlogloss:0.11582
CPU times: user 3.44 s, sys: 1.42 s, total: 4.86 s
Wall time: 32.3 s


In [41]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[155   0   4   1   1   1]
 [  1  55   4   2   1   3]
 [  0   0 415   3   2   0]
 [  0   0   8 173   0   3]
 [  0   1  38   0  87   1]
 [  1   0   4   2   0 188]]
              precision    recall  f1-score   support

           0      0.987     0.957     0.972       162
           1      0.982     0.833     0.902        66
           2      0.877     0.988     0.929       420
           3      0.956     0.940     0.948       184
           4      0.956     0.685     0.798       127
           5      0.959     0.964     0.962       195

    accuracy                          0.930      1154
   macro avg      0.953     0.895     0.918      1154
weighted avg      0.934     0.930     0.928      1154



In [42]:
answer = np.array([description[decoder[int(val)]] for val in y_test])
predss = np.array([description[decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추,딸기,시설포도,오이,토마토,파프리카
answer,,,,,,
고추,87,0,1,0,1,38
딸기,1,155,1,1,0,4
시설포도,0,1,188,2,0,4
오이,0,0,3,173,0,8
토마토,1,1,3,2,55,4
파프리카,2,0,0,3,0,415


In [43]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.9184374396084466

## risk 모델

In [49]:
# 파라미터 조정
y_all = label_risk
cnn_out_dim = 1280
description = risk_label_description
decoder = risk_label_decoder
nclass = len(description)

In [50]:
X, X_test, y, y_test = train_test_split(train_x_feature, y_all, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))

In [51]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:35883' processes=4 threads=16, memory=24.59 GiB>


[15:40:49] task [xgboost.dask]:tcp://127.0.0.1:39611 got new rank 0
[15:40:49] task [xgboost.dask]:tcp://127.0.0.1:36089 got new rank 1


[0]	train-mlogloss:1.16768
[1]	train-mlogloss:0.87360
[2]	train-mlogloss:0.67566
[3]	train-mlogloss:0.53807
[4]	train-mlogloss:0.43263
[5]	train-mlogloss:0.35197
[6]	train-mlogloss:0.28830
[7]	train-mlogloss:0.23624
[8]	train-mlogloss:0.19548
[9]	train-mlogloss:0.16285
CPU times: user 2.5 s, sys: 1.04 s, total: 3.53 s
Wall time: 22.6 s


In [52]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[765  24   0   0]
 [ 42 222   2   0]
 [ 28  20  40   0]
 [  3   7   0   1]]
              precision    recall  f1-score   support

           0      0.913     0.970     0.940       789
           1      0.813     0.835     0.824       266
           2      0.952     0.455     0.615        88
           3      1.000     0.091     0.167        11

    accuracy                          0.891      1154
   macro avg      0.920     0.587     0.637      1154
weighted avg      0.894     0.891     0.881      1154



In [53]:
answer = np.array([description[decoder[int(val)]] for val in y_test])
predss = np.array([description[decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,말기,정상,중기,초기
answer,,,,
말기,1,3,0,7
정상,0,765,0,24
중기,0,28,40,20
초기,0,42,2,222


In [54]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.6365450080987003

## disease 모델

In [60]:
# 파라미터 조정
y_all = label_disease
cnn_out_dim = 1280
description = disease_label_description
decoder = disease_label_decoder
nclass = len(description)

In [61]:
X, X_test, y, y_test = train_test_split(train_x_feature, y_all, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))

In [62]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:37607' processes=4 threads=16, memory=24.59 GiB>


[15:44:04] task [xgboost.dask]:tcp://127.0.0.1:42947 got new rank 0
[15:44:04] task [xgboost.dask]:tcp://127.0.0.1:39913 got new rank 1


[0]	train-mlogloss:1.16483
[1]	train-mlogloss:0.86842
[2]	train-mlogloss:0.66968
[3]	train-mlogloss:0.52424
[4]	train-mlogloss:0.41712
[5]	train-mlogloss:0.33324
[6]	train-mlogloss:0.26777
[7]	train-mlogloss:0.21552
[8]	train-mlogloss:0.17711
[9]	train-mlogloss:0.14604
CPU times: user 6.13 s, sys: 350 ms, total: 6.48 s
Wall time: 42.6 s


In [63]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[779   2   0   4   0   2   1   0   0   1   0   0]
 [ 24  23   1   8   0   0   2   0   0   0   0   0]
 [ 14   1  43   3   0   1   1   0   0   0   0   0]
 [ 19  12   2  24   1   2   2   0   0   0   0   0]
 [  4   0   0   0  34   0   0   0   0   0   0   0]
 [ 12   0   1   3   0  16   1   0   0   0   0   0]
 [ 20   0   0   5   0   3  34   0   0   0   0   0]
 [  1   0   0   0   0   0   1  18   0   0   0   0]
 [  3   0   0   0   0   0   0   0   8   0   0   0]
 [  8   0   0   0   0   0   0   0   0   0   0   0]
 [  5   0   0   0   0   0   0   0   0   0   1   0]
 [  4   0   0   0   0   0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

           0      0.872     0.987     0.926       789
           4      0.605     0.397     0.479        58
           5      0.915     0.683     0.782        63
           6      0.511     0.387     0.440        62
           7      0.971     0.895     0.932        38
          10      0.667     0.485     0.561        33
        

/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
answer = np.array([description[decoder[int(val)]] for val in y_test])
predss = np.array([description[decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추탄저병,다량원소결핍 (K),다량원소결핍 (N),다량원소결핍 (P),시설포도노균병,시설포도탄저병,일소피해,정상,칼슘결핍,토마토흰가루병,파프리카흰가루병
answer,,,,,,,,,,,
고추탄저병,18,0,0,0,0,0,0,1,0,0,1
다량원소결핍 (K),0,24,12,2,0,0,0,19,2,1,2
다량원소결핍 (N),0,8,23,1,0,0,0,24,0,0,2
다량원소결핍 (P),0,3,1,43,0,0,0,14,1,0,1
시설포도노균병,0,0,0,0,0,0,0,8,0,0,0
시설포도탄저병,0,0,0,0,0,8,0,3,0,0,0
일소피해,0,0,0,0,0,0,1,5,0,0,0
정상,0,4,2,0,1,0,0,779,2,0,1
축과병,0,0,0,0,0,0,0,4,0,0,0


In [65]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.5707978785841772

## 전체모델

In [66]:
# 파라미터 조정
y_all = label_disease
cnn_out_dim = 1280
description = label_description
decoder = ensemble_label_decoder
nclass = len(description)

In [67]:
X, X_test, y, y_test = train_test_split(train_x_feature, y_all, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))

In [68]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:39635' processes=4 threads=16, memory=24.59 GiB>


[15:45:44] task [xgboost.dask]:tcp://127.0.0.1:39963 got new rank 0
[15:45:44] task [xgboost.dask]:tcp://127.0.0.1:36231 got new rank 1


[0]	train-mlogloss:1.77126
[1]	train-mlogloss:1.12407
[2]	train-mlogloss:0.78695
[3]	train-mlogloss:0.58168
[4]	train-mlogloss:0.43669
[5]	train-mlogloss:0.33376
[6]	train-mlogloss:0.25914
[7]	train-mlogloss:0.19994
[8]	train-mlogloss:0.15918
[9]	train-mlogloss:0.13012
CPU times: user 8.06 s, sys: 2.16 s, total: 10.2 s
Wall time: 1min 21s


In [69]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[713  14  14  11   4   2  19   1   4   6   1   0]
 [ 27  12   5   9   0   1   3   0   0   1   0   0]
 [  7   1  42   4   0   3   4   0   1   0   0   1]
 [ 15   6   4  23   1   9   4   0   0   0   0   0]
 [ 11   0   2   0  25   0   0   0   0   0   0   0]
 [  5   4   2   7   0  14   1   0   0   0   0   0]
 [ 22   4   4   1   0   3  28   0   0   0   0   0]
 [  4   1   0   0   0   0   0  15   0   0   0   0]
 [  2   0   0   0   0   0   0   0   8   0   0   1]
 [  7   0   0   0   0   0   0   0   0   1   0   0]
 [  5   0   0   0   0   0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   0   2   0   0   2]]
              precision    recall  f1-score   support

           0      0.872     0.904     0.887       789
           4      0.286     0.207     0.240        58
           5      0.575     0.667     0.618        63
           6      0.418     0.371     0.393        62
           7      0.833     0.658     0.735        38
          10      0.438     0.424     0.431        33
        

In [70]:
answer = np.array([description[decoder[int(val)]] for val in y_test])
predss = np.array([description[decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,딸기_냉해피해_중기,딸기_냉해피해_초기,딸기_다량원소결핍 (N)_중기,딸기_다량원소결핍 (P)_말기,딸기_다량원소결핍 (P)_정상,딸기_다량원소결핍 (P)_중기,딸기_다량원소결핍 (P)_초기,딸기_딸기잿빛곰팡이병_말기,딸기_딸기흰가루병_정상,딸기_딸기흰가루병_중기,딸기_딸기흰가루병_초기,딸기_정상
answer,,,,,,,,,,,,
딸기_냉해피해_중기,28,3,0,0,0,0,0,4,4,0,1,22
딸기_냉해피해_초기,1,14,0,0,0,0,0,4,2,0,7,5
딸기_다량원소결핍 (N)_중기,0,0,15,0,0,0,0,1,0,0,0,4
딸기_다량원소결핍 (P)_말기,0,0,0,2,2,0,0,0,0,0,0,0
딸기_다량원소결핍 (P)_정상,0,0,0,1,8,0,0,0,0,0,0,2
딸기_다량원소결핍 (P)_중기,0,0,0,0,0,1,0,0,0,0,0,5
딸기_다량원소결핍 (P)_초기,0,0,0,0,0,0,1,0,0,0,0,7
딸기_딸기잿빛곰팡이병_말기,3,1,0,0,0,0,1,12,5,0,9,27
딸기_딸기흰가루병_정상,4,3,0,1,1,0,0,1,42,0,4,7


In [71]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.5075640360415982